In [18]:
filename="Isner Pospisil"
filename_csv=filename+".csv"
#correct the csv file generated by swing by comparing to the score
lst_players=filename_csv.split(".")[0].split(" ")
name1=lst_players[0]
name2=lst_players[1]
#set the time just before the first serve played
start=7*60+55

In [19]:
#COPY the 2nd sheet fo the xlsx file to a proper CSV file
import openpyxl
import csv
import pandas as pd
  
# open given workbook 
# and store in excel object 
wb = openpyxl.load_workbook(filename+".xlsx")
sheets = wb.sheetnames
sheet = wb[sheets[1]]  
  
# writer object is created
col = csv.writer(open(filename_csv,
                      'w', 
                      newline=""))
  
# writing the data in csv file
for r in sheet.rows:
    # row by row write 
    # operation is perform
    col.writerow([cell.value for cell in r])
  
# read the csv file and 
# convert into dataframe object 
df = pd.DataFrame(pd.read_csv(filename+".csv"))
  
# show the dataframe
df

,Player,Shot,Type,Stroke,Spin,Speed (KM/H),Point,Game,Set,Bounce Depth,...,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
0,Player,0,none,Feed,Flat,34.898037,3,0,0,service_box,...,deuce,near,1.092270,9.538446,2.244272,---,In,False,14:31:55,89.379997
1,Opponent,1,none,Backhand,Topspin,59.581081,3,0,0,service_box,...,ad,far,0.854255,16.037481,1.707065,down the line,In,False,14:31:56,90.180000
2,Opponent,2,none,Backhand,Topspin,59.583111,3,0,0,no_mans_land,...,ad,far,0.471261,21.687979,1.814090,cross court,In,False,14:32:00,93.879997
3,Opponent,0,none,Forehand,Topspin,35.575264,7,0,0,service_box,...,ad,far,0.438804,16.385487,1.340675,inside in,In,False,14:32:39,132.779999
4,Opponent,0,none,Feed,Topspin,32.055855,8,0,0,no_mans_land,...,deuce,near,1.170637,-0.658308,1.349950,---,Net,False,14:33:25,178.600006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Player,1,serve,Serve,Slice,210.174911,224,0,0,service_box,...,ad,near,-0.302317,-0.688414,2.535108,out wide,In,False,15:43:02,4355.979980
272,Opponent,0,none,Forehand,Flat,68.017632,225,0,0,out,...,ad,far,0.111421,19.119787,1.790266,inside out,Out,False,15:43:23,4376.979980
273,Player,1,serve,Serve,Slice,159.443024,226,0,0,service_box,...,ad,near,-0.147709,-0.821296,2.566168,out wide,In,False,15:43:45,4398.779785
274,Opponent,0,none,Backhand,Topspin,47.292587,227,0,0,service_box,...,ad_alley,far,5.100233,25.087404,1.150778,down the line,In,False,15:43:45,4399.459961


In [20]:
def get_all_shots_for_point(df):
    return (df[df["Point"].astype(int)==int(point_index)]).sort_values(by='Shot')

In [21]:
#REMOVE WARM UP shots and reindex points at 1
import pandas as pd
import numpy as np

df=pd.read_csv(filename_csv, index_col=0)
df=df.dropna(subset=["Point"])

try:
    df["Speed (KM/H)"]=round(df["Speed (KM/H)"])
except:
    pass
#remove the warm up
df=df[df["Video Time"] > start]
#reset the point at 1 (so we don't count warmup points anymore)
first_point=df.iloc[0]["Point"]
print("Point1 was actually: "+str(first_point))

#readjust point indexes
df["Point"]=df["Point"]-first_point+1
try:
    df = df.drop("Unnamed: 0", axis=1)
except:
    pass
df.to_csv(filename_csv)
df=pd.read_csv(filename_csv)
print("Point1 is now : "+str(first_point))

Point1 was actually: 22
Point1 is now : 22


In [22]:
#REMOVE ALL SHOTS where shot=0 
def remove_allshots_where_0(df):
    print("df before:{0}".format(len(df)))
    df=df[df["Shot"]>0]
    print("df after:{0}".format(len(df)))
    df.rename(columns = {'Speed (KM/H)':'Speed'," Hit (x)":"hit_x"," Hit (y)":"hit_y"," Hit (z)":"hit_z"}, inplace = True)
    dfP1=df[df.Player=="Player"]
    print("dfP1:{0}".format(len(dfP1)))
    dfP2=df[df.Player=="Opponent"]
    print("dfP2:{0}".format(len(dfP2)))
#remove_allshots_where_0(df)

In [23]:
#set the winner of the point for every shot

def set_point_winner(df, point_index: int):
    """
    Returns
    1 if player1 won the point
    0 if player 2
    -1 if player 1 hit his serve out
    -2 if player 2 hit his serve out
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    df_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = df_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = df_last_shot.iloc[0]["Player"]
    print(str_player_last_shot)
    if (df_last_shot.iloc[0]["Shot"]==1) and (last_shot_result in ["Out", "Net"]):
        if (str_player_last_shot == "Player"):
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "0" #DF
            else:
                index_winner = "-1" #Player serve out or net
        else:
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "1"#DF
            else:
                index_winner = "-2" #Opponent serve out or net
    elif (str_player_last_shot == "Player"):
        if (last_shot_result in ["In"]):
            index_winner = "1"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "0"
        else:
            index_winner = "-1"
    elif (str_player_last_shot == "Opponent"):
        if (last_shot_result in ["In"]):
            index_winner = "0"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "1"
        else:
            index_winner = "-1"
    else:
        index_winner = "-1"
    #print(str(index_winner))
    df.loc[df["Point"]==point_index,"Winner_Point"]=index_winner
    
#set_point_winner(df, 8)

In [24]:
#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
for point_index in df["Point"].unique():
    set_point_winner(df, point_index)

#SAVE FILE
df.to_csv(filename_csv)

Player
Opponent
Player
Player
Player
Player
Opponent
Opponent
Opponent
Player
Player
Player
Opponent
Player
Opponent
Player
Player
Player
Player
Opponent
Player
Opponent
Opponent
Opponent
Player
Opponent
Player
Opponent
Opponent
Player
Opponent
Player
Player
Opponent
Opponent
Opponent
Player
Opponent
Player
Opponent
Player
Player
Player
Player
Opponent
Opponent
Player
Opponent
Opponent
Opponent
Opponent
Opponent
Opponent
Player
Opponent
Opponent
Player
Opponent
Player
Player
Opponent
Opponent
Player
Player
Opponent
Player
Player
Opponent
Player
Player
Player
Player
Player
Player
Opponent
Opponent
Player
Player
Opponent
Opponent
Opponent
Opponent
Opponent
Player
Opponent
Player
Opponent
Player
Player
Opponent
Opponent
Player
Player
Opponent
Player
Opponent
Player
Opponent
Player
Player
Opponent
Player
Opponent
Opponent
Player
Opponent
Player
Opponent
Opponent
Opponent
Player
Player
Opponent
Opponent
Player
Opponent
Player
Opponent
Player
Opponent
Player
Player
Player
Player
Player
Playe

In [25]:
print(name2 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Opponent")]["Winner_Point"].value_counts()

Pospisil on SERVE, point winners:


0     24
-2    13
1     13
Name: Winner_Point, dtype: int64

In [26]:
print(name1 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Player")]["Winner_Point"].value_counts()

Isner on SERVE, point winners:


1     33
0     14
-1     5
Name: Winner_Point, dtype: int64

In [1]:
"""
check server for each serve!
check serve (if out and nb shots > 2 then it s IN) 
synchronize time between video score and file score
for each point:
	compare point winner vs score file
	- find point scored at closest time in file (max -5, +8sec)
	- if found point and error point between both sources:
		if 1 shot only
			if result == out
				result=IN
		if 2+ shots
			correct last shot from losing player => in to OUT
			remove all next shots
	- if no error
		if some earlier shots have OUT, correct them as IN
	- if not found point (+10, -10) and shot1==out or net (only serve)
		if result==in or result==out
			correct to result=out
			remove all next shots
"""
def check_server(df, point_index, df_scores):
    """
    Check if the server is right
    If not, update all shots behind serve
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    row_serve = df_all_shots_for_point[str(df_all_shots_for_point['Shot'])=="1"]
    row_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = row_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = row_last_shot.iloc[0]["Player"]
    if ()
    return df

#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
"""df_scores = pd.read_csv(filename_csv.replace("?csv","-scoreok.csv"))
for point_index in df["Point"].unique():
    check_server(df, point_index, df_scores)"""


SyntaxError: invalid syntax (<ipython-input-1-b7f22d928ad1>, line 33)